<a href="https://colab.research.google.com/github/Servat0r/ISPR-Midterms-2023/blob/master/Midterm3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Midterm 3 ISPR 2023 - Assignment 2 - Salvatore Correnti (matr. 584136)
In this assignment we will design and test on the `CIFAR-10` dataset a custom Convolutional Neural Network.

## Initial Imports
As usual, we start with a couple of cells for changing working directory and for all necessary imports before actually starting coding.

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/ISPR-Midterms-2023"

/content/drive/MyDrive/Colab Notebooks/ISPR-Midterms-2023


In [ ]:
# Below is just to make sure we can build Tensorflow with GPU and to avoid a verbose output for installation
!pip install tensorflow 1> /dev/null

Impossibile trovare il percorso specificato.


In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical

## CIFAR-10 Dataset
For training and evaluating our Convolutional Neural Network, we will use the `CIFAR-10` dataset, which is made up of $50,000$ train and $10,000$ test $32 \times 32$ RGB images, and is available in `keras` as a "built-in" dataset.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Show an image as an example
exampleImage = Image.fromarray(x_train[0], mode='RGB')
exampleImage = exampleImage.resize((64, 64))  # Just to show it better
exampleImage.show()

We now convert CIFAR-10 labels into `one-hot` format for usage with a "Sotfmax-based" CNN classifier.

In [ ]:
#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

### A quick example of CNN to fix main ideas

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(32,32,3)))
model.add(Conv2D(32, kernel_size=3, activation="relu"))
model.add(Flatten())
model.add(Dense(10, activation="softmax"))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3)

Epoch 1/3
1563/1563 [==============================] - 66s 42ms/step - loss: 2.4585 - accuracy: 0.1220 - val_loss: 2.3022 - val_accuracy: 0.1028
Epoch 2/3
1563/1563 [==============================] - 62s 40ms/step - loss: 2.2948 - accuracy: 0.1099 - val_loss: 2.3056 - val_accuracy: 0.1017
Epoch 3/3
1563/1563 [==============================] - 65s 41ms/step - loss: 2.2966 - accuracy: 0.1177 - val_loss: 2.3232 - val_accuracy: 0.1060
